In [5]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [4]:
# Read in CSV files
brands = pd.read_csv('brands.csv')
categories = pd.read_csv('categories.csv')
customers = pd.read_csv('customers.csv')
order_items = pd.read_csv('order_items.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')
staffs = pd.read_csv('staffs.csv')
stocks = pd.read_csv('stocks.csv')
stores = pd.read_csv('stores.csv')

In [6]:
# Create database connection
connection = sqlite3.connect('bike_store.db')

In [7]:
# Insert data into database
brands.to_sql('brands', connection, if_exists='replace', index=False)
categories.to_sql('categories', connection, if_exists='replace', index=False)
customers.to_sql('customers', connection, if_exists='replace', index=False)
order_items.to_sql('order_items', connection, if_exists='replace', index=False)
orders.to_sql('orders', connection, if_exists='replace', index=False)
products.to_sql('products', connection, if_exists='replace', index=False)
staffs.to_sql('staffs', connection, if_exists='replace', index=False)
stocks.to_sql('stocks', connection, if_exists='replace', index=False)
stores.to_sql('stores', connection, if_exists='replace', index=False)

3

In [11]:
query = """Select count(*) as customer_by_state, state
            from customers
            group by state;"""

In [13]:
pd.read_sql_query(query,connection)


,customer_by_state,state
0,284,CA
1,1019,NY
2,142,TX


In [18]:
query = """Select distinct first_name, last_name,phone
            from customers
            where state in ('NY')
            and phone is not null;"""


In [19]:
df = pd.read_sql_query(query,connection)
df

,first_name,last_name,phone
0,Latasha,Hays,(716) 986-3359
1,Robby,Sykes,(516) 583-7761
2,Emmitt,Sanchez,(212) 945-8823
3,Theo,Reese,(562) 215-2907
4,Janetta,Aguirre,(717) 670-2634
...,...,...,...
82,Neida,King,(442) 778-1121
83,Johana,Jacobson,(507) 407-9384
84,Mila,Moody,(248) 234-5902
85,Lamar,Bush,(716) 294-7174


Customer with phone and who do not

In [22]:
query = """SELECT
    COUNT(CASE WHEN phone IS NOT NULL THEN 1 END) AS customers_with_phone,
    COUNT(CASE WHEN phone IS NULL THEN 1 END) AS customers_without_phone
FROM
    customers;"""

In [23]:
df = pd.read_sql_query(query,connection)
df

,customers_with_phone,customers_without_phone
0,178,1267


By state

In [26]:
query = """SELECT
    COUNT(CASE WHEN phone IS NOT NULL THEN 1 END) AS customers_with_phone,
    COUNT(CASE WHEN phone IS NULL THEN 1 END) AS customers_without_phone,state
FROM
    customers
GROUP BY state;"""

In [27]:
df = pd.read_sql_query(query,connection)
df

,customers_with_phone,customers_without_phone,state
0,61,223,CA
1,87,932,NY
2,30,112,TX


By city

In [28]:
query = """SELECT
    COUNT(CASE WHEN phone IS NOT NULL THEN 1 END) AS customers_with_phone,
    COUNT(CASE WHEN phone IS NULL THEN 1 END) AS customers_without_phone,city
FROM
    customers
GROUP BY city;"""

In [29]:
df = pd.read_sql_query(query,connection)
df

,customers_with_phone,customers_without_phone,city
0,3,0,Albany
1,0,5,Amarillo
2,0,9,Amityville
3,0,5,Amsterdam
4,11,0,Anaheim
...,...,...,...
190,0,8,Woodhaven
191,0,10,Woodside
192,6,0,Yonkers
193,0,5,Yorktown Heights


Expensive product

In [30]:
query = """select product_id, max(list_price)
        from order_items;"""

In [31]:
df = pd.read_sql_query(query,connection)
df

,product_id,max(list_price)
0,155,11999.99


In [32]:
query = """select avg(discount) as avg_discount
          from order_items;"""


In [33]:
df = pd.read_sql_query(query,connection)
df

,avg_discount
0,0.105373


Total items sold

In [36]:
query = """select sum(quantity) as total_items_ordered
        from order_items;"""


In [37]:
df = pd.read_sql_query(query,connection)
df

,total_items_ordered
0,7078


In [40]:
query = """select round(avg(list_price*discount),1) as avg_discount
          from order_items;"""


In [41]:
df = pd.read_sql_query(query,connection)
df

,avg_discount
0,126.7


In [42]:
query = """ select product_id,discount
        from order_items
        where discount >= 0.2
        order by discount desc;"""

In [43]:
df = pd.read_sql_query(query,connection)
df

,product_id,discount
0,20,0.2
1,4,0.2
2,16,0.2
3,23,0.2
4,8,0.2
...,...,...
1198,236,0.2
1199,101,0.2
1200,213,0.2
1201,197,0.2


In [44]:
query = """
SELECT
    ROUND(AVG(list_price * ( 1 - discount)),2) AS avg_SALEPRICE_usd
FROM
    order_items
WHERE
	list_price >=1000;
"""


In [45]:
df = pd.read_sql_query(query, connection)
df

,avg_SALEPRICE_usd
0,2542.02


In [46]:
#Avg of the final value of sales greater than 1000
query = """
SELECT
    ROUND(AVG(list_price * ( 1 - discount)),2) AS avg_SALEPRICE_usd
FROM
    order_items
WHERE
	list_price >=1000;
"""

In [47]:
df = pd.read_sql_query(query, connection)
df

,avg_SALEPRICE_usd
0,2542.02


In [48]:
query = """
SELECT
    product_id,
    SUM(quantity) AS total_sold,
    ROUND(SUM(list_price * (1 - discount) * quantity),2) AS total_price_w_discount
FROM
    order_items
GROUP BY
    product_id
ORDER BY
    total_sold DESC;"""

In [49]:
df = pd.read_sql_query(query, connection)
df

,product_id,total_sold,total_price_w_discount
0,6,167,70371.60
1,13,157,37992.99
2,16,156,82744.62
3,23,154,41011.63
4,7,154,555558.61
...,...,...,...
302,174,1,2559.99
303,170,1,1208.99
304,167,1,712.49
305,157,1,6174.99


In [50]:
#Order that are delayed
query = """
SELECT
    *,
    CASE WHEN shipped_date > required_date THEN 1
         ELSE 0
         END AS 'shipped_late'
FROM
    orders
LIMIT 20;"""

In [51]:
df = pd.read_sql_query(query, connection)
df

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,shipped_late
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,0
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6,0
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7,0
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3,1
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6,0
5,6,94,4,2016-01-04,2016-01-07,2016-01-05,2,6,0
6,7,324,4,2016-01-04,2016-01-07,2016-01-05,2,6,0
7,8,1204,4,2016-01-04,2016-01-05,2016-01-05,2,7,0
8,9,60,4,2016-01-05,2016-01-08,2016-01-08,1,2,0
9,10,442,4,2016-01-05,2016-01-06,2016-01-06,2,6,0


In [52]:
#Counting how many are late

query = """
SELECT
    COUNT(CASE WHEN shipped_late = 1 THEN 1 ELSE NULL END) AS late,
    COUNT(CASE WHEN shipped_late = 0 THEN 1 ELSE NULL END) AS not_late
FROM (
    SELECT
        *,
        CASE WHEN shipped_date > required_date THEN 1
             ELSE 0
        END AS 'shipped_late'
    FROM
        orders
) AS subquery;"""

In [53]:
df = pd.read_sql_query(query, connection)
df

,late,not_late
0,458,1157


In [54]:
#Stock of bikes per store

query = """SELECT
    COUNT(quantity),
    stores.store_name,
    stores.state
FROM
    stocks
INNER JOIN
    stores
ON
    stocks.store_id = stores.store_id
GROUP BY stores.store_id
LIMIT 5;
"""

In [55]:
df = pd.read_sql_query(query, connection)
df

,COUNT(quantity),store_name,state
0,313,Santa Cruz Bikes,CA
1,313,Baldwin Bikes,NY
2,313,Rowlett Bikes,TX


In [56]:
query = """
SELECT
	ord.product_id,
	sum(quantity) as quant,
    ord.list_price,
    prod.product_name
FROM order_items as ord
INNER JOIN
	products as prod
ON
	ord.product_id = prod.product_id
GROUP BY
	ord.product_id
ORDER BY
	quant DESC;"""

In [57]:
df = pd.read_sql_query(query, connection)
df

,product_id,quant,list_price,product_name
0,6,167,469.99,Surly Ice Cream Truck Frameset - 2016
1,13,157,269.99,Electra Cruiser 1 (24-Inch) - 2016
2,16,156,599.99,Electra Townie Original 7D EQ - 2016
3,23,154,299.99,Electra Girl's Hawaii 1 (20-inch) - 2015/2016
4,7,154,3999.99,Trek Slash 8 27.5 - 2016
...,...,...,...,...
302,174,1,3199.99,Trek Domane SLR Disc Frameset - 2018
303,170,1,1299.99,Trek CrossRip 2 - 2018
304,167,1,749.99,Surly ECR Frameset - 2018
305,157,1,6499.99,Trek Domane SL Frameset Women's - 2018


In [58]:
#Which vendor sold the largest quantity?

query = """
SELECT COUNT(*) as sold_amount, staff_id
FROM orders
GROUP BY staff_id
ORDER BY sold_amount DESC
LIMIT 3;"""

df = pd.read_sql_query(query, connection)
df

,sold_amount,staff_id
0,553,6
1,540,7
2,184,3


In [59]:
query = """
SELECT
	COUNT(orders.order_id) as sold_amount,
	orders.staff_id,
    orders.store_id,
    staffs.first_name,
    staffs.last_name
FROM orders
LEFT JOIN
	staffs
ON
	orders.staff_id = staffs.staff_id
GROUP BY orders.staff_id
ORDER BY sold_amount DESC
LIMIT 3;"""



In [60]:
df = pd.read_sql_query(query, connection)
df


,sold_amount,staff_id,store_id,first_name,last_name
0,553,6,2,Marcelene,Boyer
1,540,7,2,Venita,Daniel
2,184,3,1,Genna,Serrano
